In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [2]:
from disease_symptoms import dataset

rash joint pain muscle pain headache
chikungunya


In [3]:
data = dataset

In [4]:
X_train = []
Y_train = []

In [5]:
for disease, symtoms in data.items():
    for symtom in symtoms:
        X_train.append(symtom)
        Y_train.append(disease)

In [6]:
label_to_id = {label : idx for idx, label in enumerate(set(Y_train))}
Y_train = [label_to_id[label] for label in Y_train]

In [7]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train)

In [8]:
import joblib
joblib.dump(tokenizer, 'disease_finding_tokenizer.joblib')

['disease_finding_tokenizer.joblib']

In [9]:
num_classes = len(set(Y_train))

In [10]:
num_classes

45

In [11]:
# Y_train_one_hot = tf.keras.utils.to_categorical(Y_train, num_classes=num_classes)

In [11]:
from tensorflow.keras.models import Sequential

In [12]:
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=50, input_length=X_train.shape[1]),
    LSTM(100, return_sequences=True),
    LSTM(50),
    Dense(num_classes, activation='softmax')
])

In [13]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [14]:
import numpy as np

In [15]:
model.fit(np.array(X_train), np.array(Y_train), epochs=100)

Epoch 1/100
6/6 [==============================] - 3s 7ms/step - loss: 3.8083 - accuracy: 0.0163
Epoch 2/100
6/6 [==============================] - 0s 7ms/step - loss: 3.8057 - accuracy: 0.0435
Epoch 3/100
6/6 [==============================] - 0s 4ms/step - loss: 3.8048 - accuracy: 0.0435
Epoch 4/100
6/6 [==============================] - 0s 7ms/step - loss: 3.8037 - accuracy: 0.0435
Epoch 5/100
6/6 [==============================] - 0s 7ms/step - loss: 3.8022 - accuracy: 0.0435
Epoch 6/100
6/6 [==============================] - 0s 7ms/step - loss: 3.8007 - accuracy: 0.0435
Epoch 7/100
6/6 [==============================] - 0s 7ms/step - loss: 3.7985 - accuracy: 0.0435
Epoch 8/100
6/6 [==============================] - 0s 11ms/step - loss: 3.7952 - accuracy: 0.0435
Epoch 9/100
6/6 [==============================] - 0s 7ms/step - loss: 3.7911 - accuracy: 0.0435
Epoch 10/100
6/6 [==============================] - 0s 6ms/step - loss: 3.7846 - accuracy: 0.0489
Epoch 11/100
6/6 [==========

In [16]:
model.save('tensorflow_model')

INFO:tensorflow:Assets written to: tensorflow_model\assets


INFO:tensorflow:Assets written to: tensorflow_model\assets


## Model Testing

In [17]:
loaded_model = tf.keras.models.load_model('tensorflow_model')

In [18]:
user_input = "I have joint pain and swelling and joint deformities."
user_input_sequence = tokenizer.texts_to_sequences([user_input])
user_input_padded = tf.keras.preprocessing.sequence.pad_sequences(user_input_sequence, maxlen=4)
prediction = loaded_model.predict(user_input_padded)
predicted_label = tf.argmax(prediction, axis=1).numpy()[0]
id_to_label = {idx : label for label, idx in label_to_id.items()}
predicted_label = id_to_label[predicted_label]
print(f"Predicted Disease : {predicted_label}")

1/1 [==============================] - 1s 514ms/step
Predicted Disease : hepatitis_c


In [19]:
import joblib
joblib.dump(id_to_label, 'id_to_label.joblib')

['id_to_label.joblib']